# BI Exercises

## Set Up

In [1]:
%load_ext sql

In [2]:
%config SqlMagic.autopandas=True

In [3]:
%config SqlMagic.displaycon=False

In [4]:
%sql sqlite:///transactions.db

In [5]:
%%sql
SELECT *
FROM transactions
LIMIT 5

Done.


,transaction_id,payment_method,seller_id,buyer_id,transaction_date,product_price
0,1294738,1,5107765,5007941,2022-05-11 15:46:38,1410.0
1,585369,4,4205200,4250752,2021-03-27 15:01:28,800.0
2,882933,4,581926,4695227,2021-10-20 14:45:11,800.0
3,1145325,3,536932,4865235,2022-03-06 17:37:04,1616.0
4,762483,5,432561,4186489,2021-08-06 11:33:12,840.0


In [6]:
%%sql
SELECT *
FROM users
LIMIT 5

Done.


,user_id,register_date
0,5042412,2022-03-01 21:26:27
1,707930,2020-09-29 08:33:26
2,5283835,2022-06-23 11:18:09
3,4195678,2021-02-24 22:12:15
4,5370530,2022-08-29 13:53:33


## Preguntas

### Agregaciones simples

#### Qué procentaje de usuarias registradas han comprado al menos un producto?

In [14]:
%%sql 
buyer_transactions <<
SELECT user_id, COUNT(DISTINCT transaction_id) AS transactions
FROM users u
LEFT JOIN transactions t
ON u.user_id = t.buyer_id
GROUP BY user_id;

Done.
Returning data to local variable buyer_transactions


In [15]:
buyer_transactions['transactions'] = list(map(lambda x: 1 if x > 0 else 0, buyer_transactions['transactions']))
round(buyer_transactions['transactions'].mean()*100)

1

#### Qué procentaje de compradoras no están registradas?

In [20]:
%%sql
users << 
select count(*) 
from users;

Done.
Returning data to local variable users


In [22]:
%%sql
buyers << 
select count(*) 
from transactions;

Done.
Returning data to local variable buyers


In [23]:
res = (1- (buyers/users))*100

In [24]:
res

,count(*)
0,33.333333


Option 2

In [29]:
%%sql

buyers_registered <<

select buyer_id, COUNT(DISTINCT register_date) as registered
FROM users as u
right join transactions as t
on u.user_id = t.buyer_id
group by buyer_id;

(sqlite3.OperationalError) RIGHT and FULL OUTER JOINs are not currently supported
[SQL: select buyer_id, COUNT(DISTINCT register_date) as registered
FROM users as u
right join transactions as t
on u.user_id = t.buyer_id
group by buyer_id;]
(Background on this error at: https://sqlalche.me/e/14/e3q8)


### Retención externa

#### Qué procentaje de usuarias han comprado al menos un producto el mes en el que se registró?

In [38]:
%%sql test <<
select 
user_id, 
strftime('%Y-%m-02', register_date),
strftime('%Y-%m-02', transaction_date),
t.*
from users u
left join transactions t
on u.user_id = t.buyer_id

Done.
Returning data to local variable test


In [39]:
test.shape

(75616, 9)

#### Cual fue el mes en el que las usuarias que se registraron convirtieron mejor a sellers? (Vendieron más items). Se mantuvo esta tendencia?

### Retención interna

#### Ha habido algún mes en el que las usuarias que compraron ese mes no volvieron a comprar el mes siguiente en mayor medida que el resto?

#### Que porcentaje de las compras cada mes están realizadas por usuarias que nunca han comprado? Y por usuarias con más de 3 meses desde su primera compra?